# **Aditya Keshav**

1. What county has the highest average weekly wage? ***(Philadelphia County)*** What is this wage? (***1266***)
2. What sector has the highest average weekly wage? (***Utilities***) What is this wage? (***1587***)
3. On a per county basis, what is the most common sector? ***(Result in CSV)***

In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
library(dplyr)
library(lubridate)
library(stringr)

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: 
Attaching package: ‘lubridate’


R[write to console]: The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [3]:
%%R
data_raw <- read.csv('/content/wages.csv')

In [4]:
%%R
data <- data_raw

In [5]:
%%R
str(data)

'data.frame':	1428 obs. of  13 variables:
 $ Area.Name                         : chr  "Greene County" "McKean County" "Armstrong County" "Warren County" ...
 $ County.Code                       : int  30 42 3 62 49 38 63 67 29 48 ...
 $ State.FIPS.Code                   : int  42 42 42 42 42 42 42 42 42 42 ...
 $ County.FIPS.Code                  : int  59 83 5 123 97 75 125 133 57 95 ...
 $ Calendar.Year                     : int  2018 2018 2018 2018 2018 2018 2018 2018 2018 2018 ...
 $ NAICS                             : chr  "61" "52" "42" "92" ...
 $ NAICS.Title                       : chr  "Educational Services" "Finance and Insurance" "Wholesale Trade" "Public Administration" ...
 $ Establishments                    : int  20 50 32 38 269 72 254 846 19 975 ...
 $ Employment                        : int  NA 276 399 587 2794 324 3261 11177 221 14692 ...
 $ Weekly.Wages                      : int  NA 795 949 861 502 652 721 1197 681 843 ...
 $ Longitude                         : num

In [6]:
%%R

data <- 
  data %>%
  select(
      County = Area.Name,
      CountyCd = County.Code,
      StateFIPS = State.FIPS.Code,
      CountyFIPS = County.FIPS.Code,
      Year = Calendar.Year,
      NAICS = NAICS,
      Sector = NAICS.Title,
      Establishments = Establishments,
      Employment = Employment,
      Wages = Weekly.Wages,
      Longitude = Longitude,
      Latitude = Latitude,
      GeoLatLong = Georeferenced.Latitude...Longitude)

In [7]:
%%R
str(data)

'data.frame':	1428 obs. of  13 variables:
 $ County        : chr  "Greene County" "McKean County" "Armstrong County" "Warren County" ...
 $ CountyCd      : int  30 42 3 62 49 38 63 67 29 48 ...
 $ StateFIPS     : int  42 42 42 42 42 42 42 42 42 42 ...
 $ CountyFIPS    : int  59 83 5 123 97 75 125 133 57 95 ...
 $ Year          : int  2018 2018 2018 2018 2018 2018 2018 2018 2018 2018 ...
 $ NAICS         : chr  "61" "52" "42" "92" ...
 $ Sector        : chr  "Educational Services" "Finance and Insurance" "Wholesale Trade" "Public Administration" ...
 $ Establishments: int  20 50 32 38 269 72 254 846 19 975 ...
 $ Employment    : int  NA 276 399 587 2794 324 3261 11177 221 14692 ...
 $ Wages         : int  NA 795 949 861 502 652 721 1197 681 843 ...
 $ Longitude     : num  -80.2 -78.6 -79.5 -79.3 -76.7 ...
 $ Latitude      : num  39.9 41.8 40.8 41.8 40.9 ...
 $ GeoLatLong    : chr  "POINT (-80.22943817 39.85480393)" "POINT (-78.5712202 41.81037074)" "POINT (-79.47316899 40.81509526)" "PO

In [8]:
%%R
nrow(data) #1428

[1] 1428


In [9]:
%%R
data <-
  data%>%
  filter(!is.na(Wages)) %>%
  filter(!is.na(Employment))

In [10]:
%%R
nrow(data) #1336

[1] 1336


In [11]:
%%R
weird <- data[F,]
indx <- str_trim(data$County) == ''
weird <- rbind(weird, data[indx,])
data <- data[!indx,]
indx <- is.na(data$County)
weird <- rbind(weird, data[indx,])
data <- data[!indx,]

indx <- str_trim(data$Sector) == ''
weird <- rbind(weird, data[indx,])
data <- data[!indx,]
indx <- is.na(data$Sector)
weird <- rbind(weird, data[indx,])
data <- data[!indx,]



In [12]:
%%R
nrow(data) #1336

[1] 1336


In [13]:
%%R
data <- 
  data %>%
  select(
      County,
      Sector,
      Employment,
      Wages)

In [14]:
%%R
str(data)

'data.frame':	1336 obs. of  4 variables:
 $ County    : chr  "McKean County" "Armstrong County" "Warren County" "Northumberland County" ...
 $ Sector    : chr  "Finance and Insurance" "Wholesale Trade" "Public Administration" "Retail Trade" ...
 $ Employment: int  276 399 587 2794 324 3261 11177 221 14692 3144 ...
 $ Wages     : int  795 949 861 502 652 721 1197 681 843 888 ...


In [25]:
%%R
dataWages <-
  data %>%
  group_by(County) %>%
  summarise(
       total = sum(Wages),
      avg = median(Wages)
  )

In [26]:
%%R
dataWages

# A tibble: 68 × 3
   County           total   avg
   <chr>            <int> <dbl>
 1 Adams County     16235   858
 2 Allegheny County 27202  1178
 3 Armstrong County 19017   785
 4 Beaver County    21006   921
 5 Bedford County   15231   738
 6 Berks County     22093  1045
 7 Blair County     18434   892
 8 Bradford County  18612   961
 9 Bucks County     24781  1023
10 Butler County    22663   987
# … with 58 more rows


In [27]:
%%R
dataCounty <-
 dataWages %>%
  group_by(County) %>%
  mutate(
      rank = row_number(-total)) %>%
  filter(rank < 4)  %>%  
  arrange(County, rank)

In [28]:
%%R
dataCounty %>%
  filter(rank == 1) %>%
  arrange(desc(avg, County))

# A tibble: 68 × 4
# Groups:   County [68]
   County              total   avg  rank
   <chr>               <int> <dbl> <int>
 1 Philadelphia County 27824  1266     1
 2 Montgomery County   28449  1255     1
 3 Allegheny County    27202  1178     1
 4 Pennsylvania        25464  1172     1
 5 Chester County      29914  1153     1
 6 Delaware County     28100  1151     1
 7 Dauphin County      22129  1144     1
 8 Lehigh County       21421  1119     1
 9 Washington County   26798  1092     1
10 Centre County       19164  1052     1
# … with 58 more rows


In [33]:
%%R
dataWages <-
  data %>%
  group_by(Sector) %>%
  summarise(
       total = sum(Wages),
      avg = median(Wages)
  )

In [34]:
%%R
dataSector <-
 dataWages %>%
  group_by(Sector) %>%
  mutate(
      rank = row_number(-total)) %>%
  filter(rank < 4)  %>%  
  arrange(Sector, rank)

In [35]:
%%R
dataSector %>%
  filter(rank == 1) %>%
  arrange(desc(avg, Sector))

# A tibble: 21 × 4
# Groups:   Sector [21]
   Sector                                           total   avg  rank
   <chr>                                            <int> <dbl> <int>
 1 Utilities                                        88954  1587     1
 2 Management of Companies and Enterprises          96871  1570     1
 3 Mining, Quarrying, and Oil and Gas Extraction    67497  1290     1
 4 Finance and Insurance                            78406  1083     1
 5 Professional, Scientific, and Technical Services 76964  1069     1
 6 Construction                                     71134  1017     1
 7 Manufacturing                                    68656  1011     1
 8 Wholesale Trade                                  72675   998     1
 9 Public Administration                            65385   977     1
10 Educational Services                             48242   954     1
# … with 11 more rows


In [36]:
%%R
dataEmployment <-
  data %>%
  filter(!Sector == "Total, All Industries")


In [37]:
%%R
str(dataEmployment)

'data.frame':	1268 obs. of  4 variables:
 $ County    : chr  "McKean County" "Armstrong County" "Warren County" "Northumberland County" ...
 $ Sector    : chr  "Finance and Insurance" "Wholesale Trade" "Public Administration" "Retail Trade" ...
 $ Employment: int  276 399 587 2794 324 3261 11177 221 14692 3144 ...
 $ Wages     : int  795 949 861 502 652 721 1197 681 843 888 ...


In [38]:
%%R
dataEmployment <-
  dataEmployment %>%
  group_by(County,Sector) %>%
  summarise(
      common = median(Employment)
  )

`summarise()` has grouped output by 'County'. You can override using the `.groups` argument.


In [39]:
%%R
dataCountySector <-
 dataEmployment %>%
  group_by(County) %>%
  mutate(
      rank = row_number(-common)) %>%
  filter(rank < 4)  %>%  
  arrange(County, rank)

In [40]:
%%R
df1 <- dataCountySector %>%
  filter(rank == 1) %>%
  arrange(desc(common, Sector))

In [41]:
%%R
df1

# A tibble: 68 × 4
# Groups:   County [68]
   County              Sector                             common  rank
   <chr>               <chr>                               <int> <int>
 1 Pennsylvania        Health Care and Social Assistance 1068610     1
 2 Philadelphia County Health Care and Social Assistance  169596     1
 3 Allegheny County    Health Care and Social Assistance  139010     1
 4 Montgomery County   Health Care and Social Assistance   79356     1
 5 Bucks County        Health Care and Social Assistance   53243     1
 6 Delaware County     Health Care and Social Assistance   44113     1
 7 Lehigh County       Health Care and Social Assistance   43063     1
 8 Lancaster County    Health Care and Social Assistance   39349     1
 9 Chester County      Health Care and Social Assistance   37022     1
10 Dauphin County      Health Care and Social Assistance   31995     1
# … with 58 more rows


In [42]:
%%R
write.csv(df1[,c("County","Sector")], '/content/result.r.csv',row.names=FALSE)